In [1]:
from datetime import datetime
from typing import Tuple, List, Union, Dict, Any
import json
import sys
import os
import uuid
import random

# Include subpath as module path
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

print("Module path: ", module_path)

from stream_handler import setup
setup()

import backend.disorientationdetectors as detectors

from backend.typealias import CoordinatePair, Sequence, Token, Pattern
import backend.datastructures as ds 
import backend.utils as utils
import backend.plotutils as plotutils
import backend.staypointdetector as spdetector
import backend.geohashsequenceinterpolator as ip
import backend.datapointstreamhandler as shandler
import backend.xchanges as xchanges
import backend.trajectorysegmentor as segmentor 
import backend.dataframeinterfaces as dfinterfaces
from django.conf import settings

import backend.models as django_models
import backend.managers as django_managers
from shapely.geometry import Polygon, Point

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.lines import Line2D
from matplotlib.dates import DateFormatter

import pandas as pd
import numpy as np
import seaborn as sns
import msgpack

# Set the following flag such that we are able to run Django inside a jupyter notebook:
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Settings
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

bits_per_char = 2
center = [55.3923666589426, 10.386784608945067]
user = "d1b0c137-1cd0-4430-ad88-4dc24c88f3ad"

Module path:  /home/nicklasxyz/Desktop/traj-dirs/traj-miner-final/RealtimeTrajectoryDataMining/rtdm


In [2]:
# Read in experimental data for case study 1
dfs = []
with open("data/case_study_1.json" ,"r") as f:
    data = json.load(f)
for item in data:
    temp_df = pd.read_json(item, orient = "split")
    for i in range(temp_df.shape[0]):
        df = pd.read_json(temp_df["df"].iloc[i] , orient = "split")
        df["external_timestamp"] = df.index
        temp_df["df"].iat[i] = df
    dfs.append(
        temp_df
    )

In [3]:
trajectories = []
for i in range(len(dfs)):
    for _, row in dfs[i].iterrows():
        df = row["df"]
        exchange = xchanges.InMemoryMessageExchangeWrapper()
        dpsh = shandler.InMemoryDataPointStreamHandler(
            **{
                "exchange": exchange,
                "max_merging_distance": 28,
                "max_radius": 100,
            },
        )
        detectors.bulk_insert_datapoint(df = df, user = user, exchange = exchange)
        trajectory = detectors.bulk_process_datapoint(
            user = user,
            exchange = exchange,
            dpsh = dpsh,
        )
        if len(trajectory) == 1:
            trajectory[0].uid = uuid.UUID(df["uid"].iloc[0])
        else:
            raise ValueError(
                "Expected only a single trajectory to be contained in the list!"
            )
        trajectories.extend(trajectory)
print(f"Read in a total of {len(trajectories)} trajectories")

Read in a total of 1 trajectories


In [4]:
dps = []
for datapoint in trajectories[0].datapoints:
    if len(datapoint.datapoints) > 0:
        for dp in datapoint.datapoints:
            dps.append(dp)
    else:
        dps.append(datapoint)

print("len(dps)                  : ", len(dps))
print("len(trajectory.datapoints): ", len(trajectories[0].datapoints))
print("Reduction                 : ", 100 * float(
    len(dps) - len(trajectories[0].datapoints) 
) / float(len(dps)), "%")

len(dps)                  :  1209
len(trajectory.datapoints):  390
Reduction                 :  67.74193548387096 %


In [5]:
map_ = plotutils.setup_map(center)
for trajectory in trajectories:
    map_ = trajectory.plot(center = center, map_ = map_)
map_

In [6]:
spd = spdetector.StayPointDetector(min_weight = 600, radius = 28, reference_user = user, **{"exchange": exchange})
spd.fit(trajectories = trajectories, user = user)
spd.plot(center = center, map_ = map_)

In [7]:
dp = trajectories[0].datapoints[0]
obj_list = []
traj_obj = django_models.Trajectory(
    start_timestamp = dp.external_timestamp,
    end_timestamp = None,
    user = user,
    tag = "raw", # Tag trajectory data as "raw"
)
traj_obj.save()
dps = trajectories[0].datapoints
traj_obj = django_models.Trajectory.objects.filter(user = user).last()  
traj_obj.start_timestamp = dps[0].external_timestamp
traj_obj.end_timestamp = dps[-1].external_timestamp
traj_obj.save(update_fields = ["start_timestamp", "end_timestamp"])
for dp in dps:
    dp_ = django_models.DataPoint(
        latitude = dp.latitude,
        longitude = dp.longitude,
        user = uuid.UUID(user), 
        label = None, 
        external_timestamp = dp.external_timestamp, 
        weight = dp.weight,
        # Nested datapoints are just saved as extra msgpack byte data
        datapoints = msgpack.dumps([_.to_dict() for _ in dp.datapoints]), 
        dx = dp.dx, 
        dt = dp.dt, 
        speed = dp.speed, 
        acceleration = dp.acceleration, 
        accuracy = dp.accuracy, 
    )
    # Associate each datapoint with the trajectory
    setattr(dp_, "trajectory", traj_obj)
    dp_.save()

In [8]:
def from_model(datapoint):
    """Convert Python DataPoint object to Django database model object.
    """
    dict_ = datapoint.__dict__.copy()
    dict_["external_timestamp"] = str(dict_["external_timestamp"]) 
    dict_["user"] = str(dict_["user"])
    dict_["trajectory"] = dict_["trajectory_id"] 
    return ds.DataPoint.from_dict(dict_)

traj = ds.Trajectory([])
traj.datapoints = [from_model(_) for _ in traj_obj.datapoints.all()]
traj.uid = str(trajectory.uid)
traj.color = trajectory.color
traj.trajectory = str(trajectory.trajectory)
traj

In [9]:
onlinets = segmentor.TrajectorySegmentor(time_period = "2.5T", **{"exchange": exchange})
onlinets._segment_trajectory_offline(user = user, spd = spd, trajectories = [traj])

Polygons:  [(<shapely.geometry.polygon.Polygon object at 0x7fc2b3c1e370>, 0), (<shapely.geometry.polygon.Polygon object at 0x7fc2b3c1e640>, 1)]
Saving 'BreakPoint' for trajectory:  53259cbd-abf7-4af3-9c09-bbfb1da601be
Saving 'BreakPoint' for trajectory:  53259cbd-abf7-4af3-9c09-bbfb1da601be
Saving 'BreakPoint' for trajectory:  53259cbd-abf7-4af3-9c09-bbfb1da601be
Saving 'BreakPoint' for trajectory:  53259cbd-abf7-4af3-9c09-bbfb1da601be
Saving 'BreakPoint' for trajectory:  53259cbd-abf7-4af3-9c09-bbfb1da601be
Saving 'BreakPoint' for trajectory:  53259cbd-abf7-4af3-9c09-bbfb1da601be
Saving 'BreakPoint' for trajectory:  53259cbd-abf7-4af3-9c09-bbfb1da601be
Saving 'BreakPoint' for trajectory:  53259cbd-abf7-4af3-9c09-bbfb1da601be
Saving 'BreakPoint' for trajectory:  53259cbd-abf7-4af3-9c09-bbfb1da601be
Saving 'BreakPoint' for trajectory:  53259cbd-abf7-4af3-9c09-bbfb1da601be
Saving 'BreakPoint' for trajectory:  53259cbd-abf7-4af3-9c09-bbfb1da601be


In [10]:
traj_obj = django_models.Trajectory.objects.filter(user = user).last()  
subtrajectories = onlinets._segment_trajectory(user, traj_obj)

In [13]:
subtrajectories

In [12]:
import random

counter = 0
map_ = plotutils.setup_map(center)
for trajectory in subtrajectories:
    random.seed(counter)
    trajectory.color = f"#{utils.color_hex(utils.rand_24_bit())}" 
    print(trajectory.color)
    map_ = trajectory.plot(center = center, map_ = map_)
    counter += 1
map_

#c53edf
#44cb63
#1cf44d
#79d67f
#78db4b
#82c9b0
#29406a
#a5cd68
#7412ca
#ed0f4c
